## Model Selection with Hyperopt & MLflow

In [2]:
import numpy as np

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK, Trials

import mlflow

### California housing dataset

- The California housing dataset is a widely used dataset in machine learning and is available in the scikit-learn library
- It contains information about housing prices in various districts of California. The dataset is often used for regression tasks to predict the median house value in a given district based on several features.

- **The California housing dataset provides the following information for each district:**

1) MedInc: Median income of households in the district.
2) HouseAge: Median age of houses in the district.
3) AveRooms: Average number of rooms per house.
4) AveBedrms: Average number of bedrooms per house.
5) Population: Total population in the district.
6) AveOccup: Average number of occupants per house.
7) Latitude: Latitude of the district's location.
8) Longitude: Longitude of the district's location.
9) MedHouseVal: Median value of houses in the district (the target variable).

- The goal of using this dataset is typically to build a regression model that can predict the median house value based on the given features.

In [3]:
X, y = fetch_california_housing(return_X_y=True)

#### Feature engineering 
##### Scale the features

Its the process of transformation numerical features in a dataset to a common scale. Its a crucial stept in data pre-procesing, help to bring the features to similar range of magnitude.

In [ ]:
from sklearn.preprocessing import StandardScaler

X.mean(axis=0)
scalar = StandardScaler()
X = scalar.fit_transform(X)
X.mean(axis=0)

#### Convert the numeric target column to discrete values